<br>

# Municípios e Comarcas

Por meio do _site_ [ListaTelefonica](https://www.tjsp.jus.br/ListaTelefonica) foi possível relevar as APIs que operam e fazer diversas pesquisas.

<br>

Michel Metran\
Data: 18.06.2024\
Atualizado em: 18.06.2024


In [ ]:
import concurrent

import pandas as pd
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from more_itertools import one

from sp_tjsp_divadmin.my_functions import strip_accents, adjust_columns
from sp_tjsp_divadmin.my_paths import output_path_tab

In [ ]:
# import os
# import pprint
# import re
# from typing import Literal

# import numpy as np
# from open_geodata import geo
# from requests_ip_rotator import ApiGateway

# from sp_tjsp_divadmin.my_functions import (
#     find_text_between_parenthesis,
#     keep_numbers,
#     remove_accents,
# )

<br>

---

## Lista de Unidades

Uma vez com a lista de todos os municípios e seus respectivos códigos, foi possível obter as informações dos fóruns por meio de outro método POST.


In [ ]:
def get_lista_unidades_tjsp(cod_municipio: int):
    """
    Pega a lista de unidades (Fóruns) de um determinado Município,
    a partir do Código do Município do TJSP
    :param cod_municipio: _description_
    :type cod_municipio: _type_
    :return: _description_
    """
    # Requests
    r: requests.Response = requests.post(
        'https://www.tjsp.jus.br/ListaTelefonica/RetornarResultadoBusca',
        json={'parmsEntrada': cod_municipio, 'codigoTipoBusca': 1},
        timeout=60,
    )

    # BS4
    soup = BeautifulSoup(r.text, 'html.parser')
    text_comarca = soup.find_all('h4')
    if text_comarca == []:
        raise Exception('Erro')

    else:
        text_comarca = one(text_comarca)
        text_comarca = text_comarca.text

        #
        comarca = text_comarca.split(' - ')[0]
        raj = text_comarca.strip().split(' - ')[-1]
        # print(raj)

        # comarca = comarca.split('está jurisdicionado à Comarca ')
        comarca = comarca.replace('Município ', '')
        comarca = comarca.replace('está jurisdicionado à Comarca', ' | ')
        comarca = comarca.replace('da Comarca', ' | ')
        # print(comarca)

        mun = comarca.strip().split(' | ')[0]
        com = comarca.strip().split(' | ')[-1]

        if mun.strip() == com.strip():
            comarca_sede = 1
        else:
            comarca_sede = 0

        # print(text_comarca)
        # print(text_comcarca.split('jurisdicionado à comarca '))

    lista_unidades = [x.text for x in soup.find_all('span')]

    return pd.DataFrame(
        {
            'id_municipio_tjsp': cod_municipio,
            'raj': raj.strip(),
            'municipio_tjsp': mun.strip(),
            'comarca_tjsp': com.strip(),
            'comarca_sede': comarca_sede,
            'unidades': lista_unidades,
        }
    )

In [ ]:
# Apenas Exemplo da Função!
id_municipio = 6504
results = get_lista_unidades_tjsp(cod_municipio=id_municipio)
display(results)

<br>

Uma vez com a função que obter os dados das Unidades e, PRINCIPALMENTE, das comarcas!
Foi possível iterar pelos 645 municípios para pegar as informações!

> IMPORTANTE: FUNÇÃO LEVA UNS 4 MINUTOS PARA RODAR!


In [ ]:
# Read Data
df_tjsp_mun = pd.read_csv(output_path_tab / 'Municipios.csv')

# Lista Municípios
list_cod_tjsp = list(df_tjsp_mun['id_municipio_tjsp'])
list_cod_tjsp[:5]

# Results
df_tjsp_mun.info()
df_tjsp_mun.head()

In [ ]:
MAX_THREADS = 5

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    temp = executor.map(get_lista_unidades_tjsp, list_cod_tjsp)
    df_com = pd.concat(list(temp), ignore_index=True)

# Results
df_com.info()
df_com.head()

Uma vez que temos a tabela de municípios e a tabela de comarcas/unidades, tratamos e juntamos


In [ ]:
# Display
display(df_tjsp_mun.head(2))
display(df_com.head(2))

In [ ]:
# Merge
df_unidades = pd.merge(
    left=df_tjsp_mun,
    right=df_com,
    left_on='id_municipio_tjsp',
    right_on='id_municipio_tjsp',
    how='inner',
    suffixes=['', '_copy'],
)

# Results
df_unidades.info()
df_unidades.head()

In [ ]:
df_unidades = df_unidades.drop(
    labels=['municipio_tjsp_copy'],
    axis='columns',
    errors='ignore',
)

# Results
df_unidades.info()
df_unidades.head()

In [ ]:
filename = 'Unidades, Municípios e Comarcas'

# Salva
df_unidades.to_csv(
    output_path_tab / f'{filename}.csv',
    index=False,
)
df_unidades.to_excel(
    output_path_tab / f'{filename}.xlsx',
    sheet_name=f'{filename}',
    index=False,
)

<br>

---

## Análises das Comarcas


Inicialmente conferi se o nome dos Municípios definido pelo TJSP correspondem ao nome das Comarcas que levantei!

Felizmente, pelo que analisei, a resposta é sim!


In [ ]:
# Read Data
df_tjsp = pd.read_csv(output_path_tab / 'Unidades, Municípios e Comarcas.csv')

# Aplica strip em todo o dataframe
df_tjsp = df_tjsp.map(lambda x: x.strip() if isinstance(x, str) else x)

# Results
df_tjsp.info()
df_tjsp.head()

In [ ]:
# Filtra Colunas
df_tjsp = df_tjsp.drop(
    labels=[
        'unidades',
        'raj',
        #'id_municipio_tjsp'
    ],
    axis='columns',
    errors='ignore',
)

df_tjsp = df_tjsp.drop_duplicates()
df_tjsp_mun = df_tjsp.copy()

# Results
df_tjsp_mun.info()
df_tjsp_mun.head()

<br>

Crio uma tabela temporária, apenas para conseguir, posteriormente, trazer os códigos do IBGE para a tabela das comarcas


In [ ]:
# Comarca
df_tjsp_com = df_tjsp[df_tjsp['comarca_sede'] == 1]

# Filtra Colunas
df_tjsp_com = df_tjsp_com.drop(
    labels=[
        #'municipio_corrigido',
        #'comarca_sede',
        #'municipio_tjsp'
        'eee'
    ],
    axis='columns',
    errors='ignore',
)

# Results
df_tjsp_com.info()
df_tjsp_com.head()

In [ ]:
# Filtra Colunas
df_tjsp_com = df_tjsp_com.drop(
    labels=['comarca_sede'],
    axis='columns',
    errors='ignore',
)

#
df_tjsp_com = df_tjsp_com.drop_duplicates()


# Renomeia
df_tjsp_com = df_tjsp_com.rename(
    mapper={
        'municipio_corrigido': 'comarca_corrigido',
        'id_municipio': 'id_comarca',
    },
    axis=1,
)

# Deleta
df_tjsp_com = df_tjsp_com.drop(
    labels=['municipio_tjsp'],
    axis='columns',
    errors='ignore',
)

# Bata Bater
df_tjsp_com = adjust_columns(df=df_tjsp_com, column_ajust='comarca_tjsp')

In [ ]:
# Results
df_tjsp_com.info()
df_tjsp_com.head()

In [ ]:
df_script1 = pd.read_csv(output_path_tab / 'Comarcas e CJs.csv')

# Aplica strip em todo o dataframe
df_script1 = df_script1.map(lambda x: x.strip() if isinstance(x, str) else x)

# Ajusta Coluna
df_script1 = adjust_columns(df=df_script1, column_ajust='comarca_tjsp')

Usando `left` eu dropo a Vila Mimosa!


In [ ]:
df_comarca = pd.merge(
    left=df_tjsp_com,
    right=df_script1,
    left_on='comarca_tjsp_temp',
    right_on='comarca_tjsp_temp',
    how='left',
    suffixes=['', '_copy'],
)

# Deleta
df_comarca = df_comarca.drop(
    labels=['comarca_tjsp_copy'],
    axis='columns',
    errors='ignore',
)

df_comarca

In [ ]:
df_comarca = df_comarca.iloc[
    df_comarca['comarca_corrigido'].str.normalize('NFKD').argsort()
]
df_comarca = df_comarca.reset_index(drop=True)
df_comarca.head()

In [ ]:
df_comarca[df_comarca['id_cj'].isna()]

In [ ]:
# df_comarca[df_comarca['id_comarca'].isna()]

In [ ]:
df_comarca = df_comarca.drop(
    labels=['comarca_tjsp_temp', 'id_municipio_tjsp'], axis='columns'
)

# Results
df_comarca.info()
df_comarca.head()

In [ ]:
filename = 'Comarcas'

# Salva
df_comarca.to_csv(
    output_path_tab / f'{filename}.csv',
    index=False,
)
df_comarca.to_excel(
    output_path_tab / f'{filename}.xlsx',
    sheet_name=f'{filename}',
    index=False,
)

In [ ]:
for file in list(output_path_tab.glob('Comarcas e CJs*')):
    print(file)
    file.unlink()

<br>

---

## Municípios

Adiciona à tabela dos municípios as comarcas


In [ ]:
# Read Data
df_tjsp = pd.read_csv(output_path_tab / 'Unidades, Municípios e Comarcas.csv')

# Aplica strip em todo o dataframe
df_tjsp = df_tjsp.map(lambda x: x.strip() if isinstance(x, str) else x)

# Results
df_tjsp.info()
df_tjsp.head()

In [ ]:
# Filtra Colunas
df_tjsp = df_tjsp.drop(
    labels=[
        'unidades',
        'raj',
        #'municipio_tjsp',
        'municipio_corrigido',
        #'id_municipio_tjsp',
    ],
    axis='columns',
    errors='ignore',
)

df_tjsp = df_tjsp.drop_duplicates()
df_tjsp_mun = df_tjsp.copy()
# df_tjsp_mun[df_tjsp_mun['municipio_tjsp'] == 'Charqueada']
df_tjsp_mun

In [ ]:
# Ajusta Coluna
df_tjsp_mun = adjust_columns(df=df_tjsp_mun, column_ajust='municipio_tjsp')

<br>

### Municípios


In [ ]:
df_script3 = pd.read_csv(output_path_tab / 'Municipios.csv')

# Aplica strip em todo o dataframe
df_script3 = df_script3.map(lambda x: x.strip() if isinstance(x, str) else x)

# Results
df_script3.info()
df_script3.head()

In [ ]:
# Ajusta Coluna
df_script3 = adjust_columns(df=df_script3, column_ajust='municipio_tjsp')

In [ ]:
# Merge
df_tjsp_merged = pd.merge(
    left=df_script3,
    right=df_tjsp_mun,
    left_on='municipio_tjsp_temp',
    right_on='municipio_tjsp_temp',
    how='left',
    suffixes=['', '_copy'],
)

# Results
df_tjsp_merged.info()
df_tjsp_merged.head()

In [ ]:
# Filtra Colunas
df_tjsp_merged = df_tjsp_merged.drop(
    labels=[
        'municipio_tjsp_temp',
        'id_municipio_tjsp_copy',
        'municipio_tjsp_copy',
        'id_municipio_copy',
    ],
    axis='columns',
    errors='ignore',
)

# Results
df_tjsp_merged.info()
df_tjsp_merged.head()

Uma vez que adicionei o nome da Comarca...\
Desejo converter para número...

Para isso eu leio a tabela de comarcas.


<br>

### Comarcas


In [ ]:
# Read Data
df_tjsp_com = pd.read_csv(output_path_tab / 'Comarcas.csv')

# Results
df_tjsp_com.info()
df_tjsp_com.head()

In [ ]:
df_tjsp_com = adjust_columns(df=df_tjsp_com, column_ajust='comarca_tjsp')
df_tjsp_com

In [ ]:
df_tjsp_merged = adjust_columns(df=df_tjsp_merged, column_ajust='comarca_tjsp')
df_tjsp_merged

In [ ]:
df_municipio = pd.merge(
    left=df_tjsp_merged,
    right=df_tjsp_com,
    left_on='comarca_tjsp_temp',
    right_on='comarca_tjsp_temp',
    how='left',
    suffixes=['', '_copy'],
)

df_municipio = df_municipio.drop(
    labels=[
        'comarca_tjsp_temp',
        'comarca_corrigido',
        'comarca_tjsp_copy',
        'comarca_tjsp',
    ],
    axis='columns',
)

df_municipio

In [ ]:
filename = 'Municipios'

# Salva
df_municipio.to_csv(output_path_tab / f'{filename}.csv', index=False)
df_municipio.to_excel(
    output_path_tab / f'{filename}.xlsx', sheet_name=f'{filename}', index=False
)